In [4]:
import sys
import os
from glob import glob
sys.path.append("../")
from NEATModels import NEATStaticDetection, nets
from NEATModels.Staticconfig  import StaticNeatConfig
from NEATUtils import helpers
from NEATUtils.helpers import save_json, load_json
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

Using TensorFlow backend.


In [5]:
#Input the training directory containing training image patches and labels 
StaticCenterTrainDataDir = '/home/kapoorlab/StaticCenterTrainData/'
#Directory containing the train category json, model parameter and weight vector file
ModelDir = '/home/kapoorlab/NEATModel/'
#Model name based on wether it is residual or sequntial ONEAT network
Model_Name = 'OSStaticONEAT.h5'
StaticCategoriesJson = ModelDir + 'StaticCategories.json'
KeyCategories = load_json(StaticCategoriesJson)

StaticCordJson = ModelDir + 'StaticCord.json'
KeyCord = load_json(StaticCordJson)

In [6]:
#Neural network parameters
#For ORNET use residual = True and for OSNET use residual = False
residual = False
#NUmber of starting convolutional filters, is doubled down with increasing depth
startfilter = 32
#CNN network start layer, mid layers and lstm layer kernel size
start_kernel = 3
mid_kernel = 3
#Network depth has to be 9n + 2, n= 3 or 4 is optimal for Notum dataset
depth = 29
#Training epochs, longer the better zith proper chosen leqrning rate
epochs = 150
#Size of the gradient descent length vector, start small and use callbacs to get smaller when reaching the minima
learning_rate = 1.0E-4
#For stochastic gradient decent, the batch size used for computing the gradients
batch_size = 10
# use softmax for single event per box, sigmoid for multi event per box
multievent = False
# Grid and number of boxes for yolo
gridX = 1
gridY = 1
nboxes = 5
#Weightage to co-ordinate loss term
lambdacord = 5

In [3]:
config = StaticNeatConfig(residual = residual, depth = depth, start_kernel = start_kernel, mid_kernel = mid_kernel,
                         startfiler = startfilter, gridX = gridX, gridY = gridY, nboxes = nboxes, lambdacord = lambdacord,
                         epochs = epochs, categories = len(KeyCategories), learning_rate = learning_rate,
                         batch_size = batch_size, ModelName = Model_Name, multievent = multievent)

config_json = config.to_json()
show = True
print(config)
save_json(config_json, Model_dir + Model_Name + '_Parameter.json')

StaticNeatConfig(ModelName='StaticOSNETd29K3.h5', batch_size=100, depth=29, epochs=250, learning_rate=0.0001, mid_kernel=3, residual=True, simple=False, start_kernel=3, startfilter=32)


In [ ]:
Train = NEATStaticDetection(config, TrainDataDir, KeyCategories, KeyCord, Model_dir, Model_Name, show = show)

Train.loadData()

Train.TrainModel()
